In [107]:
import pandas as pd
import numpy as np
from prophet import Prophet
from favorita import functions
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning, )
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

# Get the best params from the Grid Search

In [108]:
df_params = pd.read_csv('/home/emilien/code/EmilienStoret/favorita/Data/best_params_dataframe.csv')
df_best_params = df_params[df_params['mdape']==df_params['mdape'].min()]
df_best_params

,Unnamed: 0,changepoint_prior_scale,fourier_order,prior_scale_dcoilwtico,prior_scale_is_holiday,prior_scale_onpromotion,seasonality_mode,seasonality_prior_scale,mdape
263,263,0.08,7,0.01,0.1,0.01,multiplicative,0.11,0.117857


In [109]:
index = df_best_params.index[0]
index

263

# Get the train and test datasets for all the stores

In [110]:
df_main = pd.read_csv('/home/emilien/code/EmilienStoret/favorita/Data/df_main_V4.csv')
df_main.tail()

,id,date,store_nbr,family,sales,onpromotion,city,holidays,holidays_nat,is_holiday,dcoilwtico
3008275,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,0,0,0,47.57
3008276,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,0,0,0,47.57
3008277,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,0,0,0,47.57
3008278,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,0,0,0,47.57
3008279,3000887,2017-08-15,9,SEAFOOD,16.000,0,Quito,0,0,0,47.57


In [111]:
df_main_stores = df_main.groupby(['date','store_nbr']).agg({'sales':'sum',
                                                         'onpromotion':'sum',
                                                         'is_holiday':'mean',
                                                         'dcoilwtico':'mean'})
df_main_stores.reset_index(inplace=True)

for i in range(1, df_main_stores['store_nbr'].max()+1):
    df_loop=df_main_stores[df_main_stores['store_nbr']==i]
    df_loop.reset_index(inplace=True, drop=True)
    df_loop=df_loop.rename(columns={'date':'ds', 'sales':'y'})
    df_loop.drop(columns='store_nbr', inplace=True)
    globals()['df_store_'+str(i)] = df_loop
    globals()['df_train_store_'+str(i)]=df_loop[df_loop['ds']<'2017-04-01']
    globals()['df_test_store_'+str(i)]=df_loop[(df_loop['ds']>='2017-04-01')&(df_loop['ds']<'2017-07-01')]
    globals()['df_test_store_'+str(i)].reset_index(inplace=True, drop=True)
    
df_store_54.head()

,ds,y,onpromotion,is_holiday,dcoilwtico
0,2013-01-01,0.000,0,1.0,93.14
1,2013-01-02,4973.285,0,0.0,93.14
2,2013-01-03,3901.570,0,0.0,92.97
3,2013-01-04,3266.966,0,0.0,93.12
4,2013-01-05,4394.549,0,1.0,93.12


# Drop the outliers

In [112]:
for i in range(1, df_main_stores['store_nbr'].max()+1):
    q1, q3 = np.percentile(globals()['df_train_store_'+str(i)]['y'],[25,75])
    IQR = q3-q1
    df_outliers = globals()['df_train_store_'+str(i)][globals()['df_train_store_'+str(i)]['y']>3*IQR]
    globals()['df_train_store_'+str(i)].drop(index=df_outliers.index, inplace=True)

/tmp/ipykernel_32628/1933209214.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()['df_train_store_'+str(i)].drop(index=df_outliers.index, inplace=True)


# Train prophet

In [114]:
from tqdm import tqdm

In [115]:
for i in tqdm(range(1, df_main_stores['store_nbr'].max()+1)):
    
    # Train the model
    df_train = globals()['df_train_store_'+str(i)]

    m = Prophet(changepoint_prior_scale = df_best_params['changepoint_prior_scale'][index],
                seasonality_prior_scale = df_best_params['seasonality_prior_scale'][index],
                seasonality_mode = df_best_params['seasonality_mode'][index],
                weekly_seasonality=True,
                daily_seasonality = False,
                yearly_seasonality = True,
                interval_width=0.95)
    m.add_regressor('is_holiday', prior_scale = df_best_params['prior_scale_is_holiday'][index])
    m.add_regressor('onpromotion', prior_scale = df_best_params['prior_scale_onpromotion'][index])
    m.add_regressor('dcoilwtico', prior_scale = df_best_params['prior_scale_dcoilwtico'][index])
    m.add_seasonality(name='monthly', period=30.417, fourier_order=df_best_params['fourier_order'][index])
    model=m.fit(df_train[['ds', 'y', 'is_holiday', 'onpromotion', 'dcoilwtico']])

    #forecast from prophet
    df_loop = globals()['df_store_'+str(i)]
    future=m.make_future_dataframe(periods=91, freq= 'D')
    future.insert(1, 'is_holiday', df_loop['is_holiday'])
    future.insert(2, 'onpromotion', df_loop['onpromotion'])
    future.insert(3, 'dcoilwtico', df_loop['dcoilwtico'])
    globals()['df_store_'+str(i)+'_pred'] = m.predict(future)
    

100%|███████████████████████████████████████████████████████████████████████████████████| 54/54 [00:52<00:00,  1.03it/s]


In [ ]:
df_store_2_pred.head()

In [ ]:
# Get MDAPE on Q2 2017

d_score={'store_nbr':[], 'MAPE': []}

for i in range(1, df_main_stores['store_nbr'].max()+1):
    df_loop = globals()['df_store_'+str(i)+'_pred']
    globals()['df_store_'+str(i)+'_pred_only']=df_loop[(df_loop['ds']>='2017-04-01')&(df_loop['ds']<'2017-07-01')]
    globals()['df_store_'+str(i)+'_pred_only'].reset_index(inplace = True, drop =True)
    mdape = abs((globals()['df_store_'+str(i)+'_pred_only']['yhat']-globals()['df_test_store_'+str(i)]['y'])
                /globals()['df_test_store_'+str(i)]['y'])
    d_score['store_nbr'].append(i)
    d_score['MAPE'].append(mdape.mean())
    
d_score = pd.DataFrame(d_score)


In [ ]:
d_score

# Compare the results with the baseline

In [ ]:
df_baseline_score = pd.read_csv('/home/emilien/code/EmilienStoret/favorita/Data/baseline_score.csv')
df_baseline_score.rename(columns={"store":"store_nbr"}, inplace=True)


In [ ]:
d_score['store_nbr'].head()

In [ ]:
df_baseline_score['store_nbr'].head()

In [ ]:
df_score = d_score.merge(df_baseline_score, on='store_nbr', how='left')
df_score.drop(columns='Unnamed: 0', inplace = True)

In [ ]:
df_score

In [ ]:
#df_score.drop(columns='Unnamed: 0', inplace = True)
df_score['is_better']=df_score['MAPE']-df_score['baseline_score']
df_score['is_better']=df_score['is_better'].apply(lambda x: 1 if x<0 else 0)
df_score

In [ ]:
df_score['is_better'].sum()